In [1]:
import numpy as np

class FingerGripDetector:
    def __init__(self, num_sensors=1062, contact_threshold=50, grip_force_threshold=60):
        """
        num_sensors: 전체 센서 개수 (예: 1062)
        contact_threshold: 센서를 활성화된 것으로 간주할 임계값
        grip_force_threshold: 파지 여부를 판단할 평균 힘(force) 임계값
        """
        self.num_sensors = num_sensors
        self.contact_threshold = contact_threshold
        self.grip_force_threshold = grip_force_threshold

    def update(self, tactile_values):
        # 활성화된 센서만 추출
        active_sensors = tactile_values > self.contact_threshold
        active_ratio = np.sum(active_sensors) / self.num_sensors

        # 접촉(contact) 여부
        contact_detected = active_ratio > 0.01  # 1% 이상의 센서가 활성화되면 접촉으로 판단

        # Grip 여부: 활성 센서 평균 힘(force)이 일정 임계값 이상이면 Grip
        avg_force_active = np.mean(tactile_values[active_sensors]) if contact_detected else 0
        grip_detected = contact_detected and avg_force_active > self.grip_force_threshold

        return contact_detected, grip_detected, active_ratio, avg_force_active


# -------------------- 사용 예시 --------------------
file_path = "/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/data/episode_0002/tactiles/001016_left_tactile.npy"
tactile_data = np.load(file_path)  # shape: (1062,)

detector = FingerGripDetector(contact_threshold=50, grip_force_threshold=60)
contact, grip, ratio, avg_force = detector.update(tactile_data)

print(f"Contact Detected: {contact} (Active Sensor Ratio: {ratio*100:.2f}%)")
print(f"Grip Detected: {grip} (Avg Force of Active Sensors: {avg_force:.2f})")


Contact Detected: True (Active Sensor Ratio: 3.01%)
Grip Detected: True (Avg Force of Active Sensors: 742.59)


In [2]:
import numpy as np
from collections import deque
from pydub import AudioSegment
from pydub.playback import play
import threading

class RobustGripDetector:
    def __init__(self, touch_dict, contact_threshold=50, grip_force_threshold=60,
                 min_fingers=2, grip_hold_time=0.0, fps=30):
        self.contact_threshold = contact_threshold
        self.grip_force_threshold = grip_force_threshold
        self.min_fingers = min_fingers
        self.history = deque(maxlen=int(grip_hold_time * fps)) if grip_hold_time > 0 else None
        self.finger_indices = self._create_finger_indices(touch_dict)

    def _create_finger_indices(self, touch_dict):
        indices = {}
        start = 0
        for name, count in touch_dict.items():
            indices[name] = range(start, start + count)
            start += count
        return indices

    def update(self, tactile_values):
        active_sensors = tactile_values > self.contact_threshold
        
        # 손가락별 활성화 비율 계산 (손바닥 제외)
        active_fingers = []
        for name, idx in self.finger_indices.items():
            if "palm" in name:  # 손바닥 제외
                continue
            ratio = np.sum(active_sensors[idx]) / len(idx)
            if ratio > 0.05:
                finger_name = name.split("_")[0]
                if finger_name not in active_fingers:
                    active_fingers.append(finger_name)

        # Contact 여부
        contact_detected = len(active_fingers) > 0
        avg_force_active = np.mean(tactile_values[active_sensors]) if contact_detected else 0

        # Grip 여부
        if self.history is not None:
            self.history.append(avg_force_active)
            grip_detected = (
                contact_detected
                and len(active_fingers) >= self.min_fingers
                and len(self.history) == self.history.maxlen
                and np.min(self.history) > self.grip_force_threshold
            )
        else:
            # grip_hold_time=0일 경우, 히스토리 없이 즉시 판정
            grip_detected = (
                contact_detected
                and len(active_fingers) >= self.min_fingers
                and avg_force_active > self.grip_force_threshold
            )

        return contact_detected, grip_detected, active_fingers, avg_force_active


In [ ]:
class StereoSoundFeedbackManager:
    def __init__(self, grip_sound_path):
        self.prev_grip_state = {"left": False, "right": False}
        self.grip_sound = AudioSegment.from_file(grip_sound_path)  # 사운드 로드

    def play_sound(self, hand="right"):
        sound = self.grip_sound

        # 오른손이면 오른쪽 채널 강조, 왼손이면 왼쪽 채널 강조
        if hand == "right":
            sound = sound.pan(1.0)   # 오른쪽으로 완전히 이동
        elif hand == "left":
            sound = sound.pan(-1.0)  # 왼쪽으로 완전히 이동

        threading.Thread(target=play, args=(sound,)).start()  # 비동기 재생

    def update(self, grip_detected, hand="right"):
        # Grip False → True로 전이될 때만 재생
        if grip_detected and not self.prev_grip_state[hand]:
            self.play_sound(hand)
        self.prev_grip_state[hand] = grip_detected



In [7]:
if __name__ == "__main__":
    touch_dict = {
        "fingerone_tip_touch": 9,
        "fingerone_top_touch": 96,
        "fingerone_palm_touch": 80,
        "fingertwo_tip_touch": 9,
        "fingertwo_top_touch": 96,
        "fingertwo_palm_touch": 80,
        "fingerthree_tip_touch": 9,
        "fingerthree_top_touch": 96,
        "fingerthree_palm_touch": 80,
        "fingerfour_tip_touch": 9,
        "fingerfour_top_touch": 96,
        "fingerfour_palm_touch": 80,
        "fingerfive_tip_touch": 9,
        "fingerfive_top_touch": 96,
        "fingerfive_middle_touch": 9,
        "fingerfive_palm_touch": 96,
        "palm_touch": 112
    }

    file_path = "/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/data/episode_0001/tactiles/000524_right_tactile.npy"
    tactile_data = np.load(file_path)

    detector = RobustGripDetector(
        touch_dict=touch_dict,
        contact_threshold=30,
        grip_force_threshold=40,
        min_fingers=2,
        grip_hold_time=0.0
    )

    sound_manager = StereoSoundFeedbackManager(grip_sound_path="/home/scilab/Documents/teleoperation/avp_teleoperate/hapticfeedback/sounddata/bell-notification-337658.mp3")

    # 오른손 grip 테스트
    contact, grip, active_fingers, avg_force = detector.update(tactile_data)
    sound_manager.update(grip_detected=grip, hand="left")

    print(f"Contact Detected: {contact}")
    print(f"Grip Detected: {grip}")
    print(f"Active Fingers: {active_fingers}")
    print(f"Average Force (active sensors): {avg_force:.2f}")

Contact Detected: True
Grip Detected: True
Active Fingers: ['fingertwo', 'fingerthree']
Average Force (active sensors): 199.71
